In [30]:
# Matrix factorization with regularization
# CSCI 6140 HW4

import matplotlib.pyplot as plt
import numpy as np

def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    
    reg_term = reg * Ui
    error_term = (Yij - (np.dot(Ui, Vj))) * Vj
    gradient = reg_term - error_term 
    return (eta * gradient)


def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    reg_term = reg * Vj
    error_term = (Yij - (np.dot(Ui, Vj))) * Ui
    gradient = reg_term - error_term 
    return (eta * gradient)

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    sqrd_error = 0
    for i in range(len(Y)):
        (i, j, Y_ij) = Y[i]
        sqrd_error += 0.5 * (Y_ij - (np.dot(U[i - 1], V[j - 1])))**2
    val = sqrd_error / (len(Y))
    return val


def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
   
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    # U matrix initialized to random numbers
    U = np.random.uniform(-0.5, 0.5, (M, K))
    # V matrix initialized to random numbers
    V = np.random.uniform(-0.5, 0.5, (N, K))
    
    # MSE of the first epoch
    curr_MSE_err = get_err(U, V, Y)
    
    # list of indices
    indices = list(range(len(Y)))
    
    # on shuffling indices
    shuffled_indices = np.random.permutation(indices)
       
    # computing new U and V using gradient descent    
    for i in range(len(Y)):
        i = shuffled_indices[i]
        (i, j, Y_ij) = Y[i]
        U[i - 1] = U[i - 1] - grad_U(U[i - 1], Y_ij, V[j - 1], reg, eta)
        V[j - 1] = V[j - 1] - grad_V(V[j - 1], Y_ij, U[i - 1], reg, eta)
        
        
    # calculating MSE for new U and Y matrix generated    
    new_MSE_err = get_err(U, V, Y)
    difference = (curr_MSE_err - new_MSE_err)
    curr_MSE_err = new_MSE_err

    epoch_cnt = 1
    decrease = difference
    while (epoch_cnt < max_epochs and decrease > eps * difference):
        
         # on shuffling indices
        indices = list(range(len(Y)))
        shuffled_indices = np.random.permutation(indices)
        
        # computing new U and V using gradient descent    
        for i in range(len(Y)):
            i = shuffled_indices[i]
            (i, j, Y_ij) = Y[i]
            U[i - 1] = U[i - 1] - grad_U(U[i - 1], Y_ij, V[j - 1], reg, eta)
            V[j - 1] = V[j - 1] - grad_V(V[j - 1], Y_ij, U[i - 1], reg, eta)
        
        #calculating new MSE
        new_MSE_err = get_err(U, V, Y)
        decrease = curr_MSE_err - new_MSE_err
        curr_MSE_err = new_MSE_err
        epoch_cnt += 1
        
    return (U, V, curr_MSE_err)




def main():
    Y_train = np.loadtxt('/Users/ruchitha/Desktop/ML-Assignments/HW4/HW4 3/train.txt').astype(int)
    Y_test = np.loadtxt('/Users/ruchitha/Desktop/ML-Assignments/HW4/HW4 3/test.txt').astype(int)
    M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
    N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
    Ks = [10,20,30,50,100]

    
    regs = [10**-4, 10**-3, 10**-2, 10**-1, 1]
    eta = 0.03 # learning rate
    E_ins = []
    E_outs = []

    # Use to compute Ein and Eout
    
    for reg in regs:
        E_ins_for_lambda = []
        E_outs_for_lambda = []
        
        for k in Ks:
            
            U,V, e_in = train_model(M, N, k, eta, reg, Y_train)
            E_ins_for_lambda.append(e_in)
            eout = get_err(U, V, Y_test)
            E_outs_for_lambda.append(eout)
            
        E_ins.append(E_ins_for_lambda)
        E_outs.append(E_outs_for_lambda)

      
    # Plot values of E_in across k for each value of lambda
    for i in range(len(regs)):
        plt.plot(Ks, E_ins[i], label='$E_{in}, \lambda=$'+str(regs[i]))
    plt.title('$E_{in}$ vs. K')
    plt.xlabel('K')
    plt.ylabel('Error')
    plt.legend()
    plt.savefig('2e_ein.png')	
    plt.clf()

#     # Plot values of E_out across k for each value of lambda
#     for i in range(len(regs)):
#         plt.plot(Ks, E_outs[i], label='$E_{out}, \lambda=$'+str(regs[i]))
#     plt.title('$E_{out}$ vs. K')
#     plt.xlabel('K')
#     plt.ylabel('Error')
#     plt.legend()
#     plt.savefig('2e_eout.png')

if __name__ == "__main__":
    main()

<Figure size 432x288 with 0 Axes>